In [11]:
import numpy as np
import torch
from diffusers.evaluation.conditional_evaluation import ConditionalEvaluation

In [2]:

latent_cond_rke_p = '/media/student/data/rke_guidance/dog_text_to_image_prompts_ambient_experiment/sdxl_cond-rke_0.02_5_20250513_2053_simga=(0.8, 0.6)/clip.npz'
clip_cond_rke_p = '/media/student/data/rke_guidance/ambient_space_guidance/sdxl_dog_ambient/dog_text_to_image_prompts _sdxl__cond-rke_0.05_5.0-20250513T123551Z-1-001/dog_text_to_image_prompts _sdxl__cond-rke_0.05_5.0/clip.npz'

l_cond_rke = np.load(latent_cond_rke_p)['clip_features'][:77]
c_cond_rke = np.load(clip_cond_rke_p)['clip_features']

In [26]:
eval_model = ConditionalEvaluation(sigma=(0, 0))

print(torch.exp(eval_model.compute_entropy(l_cond_rke, 1, compute_kernel=True, algorithm='kernel', kernel='cosine', computation_method='vendi', top_k=None)))

tensor(4.4266, device='cuda:0', dtype=torch.float64)


In [24]:
print(torch.exp(eval_model.compute_entropy(c_cond_rke, 1, compute_kernel=True, algorithm='kernel', kernel='cosine', computation_method='vendi', top_k=None)))

tensor(5.3350, device='cuda:0', dtype=torch.float64)


In [27]:
latent_cond_rke_p = '/media/student/data/rke_guidance/dog_text_to_image_prompts_ambient_experiment/sdxl_cond-rke_0.02_5_20250513_2053_simga=(0.8, 0.6)/dino.npz'
clip_cond_rke_p = '/media/student/data/rke_guidance/ambient_space_guidance/sdxl_dog_ambient/dog_text_to_image_prompts _sdxl__cond-rke_0.05_5.0-20250513T123551Z-1-001/dog_text_to_image_prompts _sdxl__cond-rke_0.05_5.0/dino.npz'

l_cond_rke = np.load(latent_cond_rke_p)['dino_features'][:77]
c_cond_rke = np.load(clip_cond_rke_p)['dino_features']

In [28]:
print(torch.exp(eval_model.compute_entropy(l_cond_rke, 1, compute_kernel=True, algorithm='kernel', kernel='cosine', computation_method='vendi', top_k=None)))

tensor(25.2212, device='cuda:0', dtype=torch.float64)


In [29]:
print(torch.exp(eval_model.compute_entropy(c_cond_rke, 1, compute_kernel=True, algorithm='kernel', kernel='cosine', computation_method='vendi', top_k=None)))

tensor(13.3316, device='cuda:0', dtype=torch.float64)


In [39]:
import torch
import torch.nn.functional as F
from collections import defaultdict


def compute_cluster_similarities(features: torch.Tensor, 
                               clusters: list) -> dict:
    """
    Compute average pairwise similarity within each cluster.
    
    Args:
        features: Tensor of shape (batch_size, feature_dim)
        clusters: List of cluster IDs (length = batch_size)
    
    Returns:
        Dictionary {cluster_id: average_similarity}
    """
    # Input validation
    assert len(features) == len(clusters), "Features and clusters must have same length"
    if len(features) < 2:
        return {}
    
    # Normalize features
    features = F.normalize(features, p=2, dim=1)
    
    # Compute similarity matrix
    sim_matrix = torch.mm(features, features.T)  # [batch_size, batch_size]
    
    # Calculate within-cluster similarities
    results = defaultdict(list)
    batch_size = len(clusters)
    
    for i in range(batch_size):
        for j in range(i+1, batch_size):  # Upper triangle only
            if clusters[i] == clusters[j]:
                results[clusters[i]].append(sim_matrix[i,j].item())
    
    return {k: sum(v)/len(v) for k,v in results.items() if len(v) > 0}


In [47]:
# Compute score (range [-1, 1], where 1 = identical features)

clusters = [
    # Cluster 0: Sitting dogs (no location context)
    0, 0, 0, 0, 0, 0, 0, 0,
    
    # Cluster 1: Standing beside cherry blossom (East Asia)
    1,
    
    # Cluster 2: Standing beside Gothic cathedral (Europe)
    2, 2, 2, 2, 2, 2, 2, 2,
    
    # Cluster 3: Standing beside village hut (Kenya/Africa)
    3, 3, 3, 3, 3, 3, 3, 3,
    
    # Cluster 4: Chasing birds (Europe streets)
    4, 4, 4, 4, 4, 4, 4, 4,
    
    # Cluster 5: Chasing birds (Nile River/Africa)
    5, 5, 5, 5, 5, 5, 5, 5,
    
    # Cluster 6: Guarding barn (America)
    6, 6, 6, 6, 6, 6, 6, 6,
    
    # Cluster 7: Guarding livestock (Africa)
    7, 7, 7, 7, 7, 7, 7, 7,
    
    # Cluster 8: Guarding palace (East Asia)
    8, 8, 8, 8, 8, 8, 8, 8,
    
    # Cluster 9: Hiding in grass (Africa)
    9, 9, 9, 9, 9, 9, 9, 9,
    
    # Cluster 10: Temple courtyard (East Asia)
    10, 10, 10, 10
]
similarity_score = compute_cluster_similarities(torch.tensor(l_cond_rke), clusters)
print(f"In-batch similarity score: {similarity_score}")
similarity_score.values()

In-batch similarity score: {0: 0.3766195814854997, 2: 0.7493791352297682, 3: 0.4886314957511692, 4: 0.5728345473833436, 5: 0.40428764183557914, 6: 0.600555954984211, 7: 0.3635234456415925, 8: 0.7444103819783254, 9: 0.7736855598994824, 10: 0.6217056887921234}


dict_values([0.3766195814854997, 0.7493791352297682, 0.4886314957511692, 0.5728345473833436, 0.40428764183557914, 0.600555954984211, 0.3635234456415925, 0.7444103819783254, 0.7736855598994824, 0.6217056887921234])

In [48]:
np.mean([0.3766195814854997, 0.4886314957511692, 0.5728345473833436, 0.40428764183557914, 0.600555954984211, 0.3635234456415925, 0.7444103819783254, 0.7736855598994824, 0.6217056887921234])

0.5495838108612585

In [49]:
similarity_score = compute_cluster_similarities(torch.tensor(c_cond_rke), clusters)
print(f"In-batch similarity score: {similarity_score}")
similarity_score.values()

In-batch similarity score: {0: 0.8974452452676359, 2: 0.9311956053519819, 3: 0.7580989492752886, 4: 0.8922430134246893, 5: 0.8697051569943206, 6: 0.8509554957221214, 7: 0.8879562498082236, 8: 0.9564437232131068, 9: 0.7887484041511701, 10: 0.8514224023067491}


dict_values([0.8974452452676359, 0.9311956053519819, 0.7580989492752886, 0.8922430134246893, 0.8697051569943206, 0.8509554957221214, 0.8879562498082236, 0.9564437232131068, 0.7887484041511701, 0.8514224023067491])

In [ ]:
[0.8974452452676359, 0.7580989492752886, 0.8922430134246893, 0.8697051569943206, 0.8509554957221214, 0.8879562498082236, 0.9564437232131068, 0.7887484041511701, 0.8514224023067491]